In [1]:
!conda env list

# conda environments:
#
base                     /home/chris/anaconda3
basic                    /home/chris/anaconda3/envs/basic
torch                    /home/chris/anaconda3/envs/torch
trans                 *  /home/chris/anaconda3/envs/trans
trans2                   /home/chris/anaconda3/envs/trans2



In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'available device: {device}')
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

available device: cuda


# Prepare data & data loader

In [3]:
mnist_train = torchvision.datasets.MNIST(root='../dat/',
                                         train=True,
                                         transform=transforms.ToTensor(),
                                         download=True)
mnist_test = torchvision.datasets.MNIST(root='../dat/',
                                        train=False,
                                        transform=transforms.ToTensor(),
                                        download=True)

In [4]:
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=True)

# Define my Neural Network

In [5]:
class myNN(torch.nn.Module):
    def __init__(self):
        super(myNN, self).__init__()

        self.debug = False
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = torch.nn.ReLU()
        self.conv12 = torch.nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.relu12 = torch.nn.ReLU()
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = torch.nn.ReLU()
        self.conv22 = torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu22 = torch.nn.ReLU()
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu3 = torch.nn.ReLU()
        self.conv32 = torch.nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.relu32 = torch.nn.ReLU()
        self.conv33 = torch.nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.relu33 = torch.nn.ReLU()

        self.fc = torch.nn.Linear(7 * 7 * 128, 10, bias=True)
        self.fc_bn = torch.nn.BatchNorm1d(10)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv12(out)
        out = self.relu12(out)
        out = self.pool1(out)

        out = self.conv2(out)
        out = self.relu2(out)
        out = self.conv22(out)
        out = self.relu22(out)
        out = self.pool2(out)

        out = self.conv3(out)
        out = self.relu3(out)
        out = self.conv32(out)
        out = self.relu32(out)
        out = self.conv33(out)
        out = self.relu33(out)

        out = out.view(out.size(0), -1)
        out = self.fc(out)
        out = self.fc_bn(out)
        return out

# Set training protocols

In [6]:
model = myNN().to(device=device)
learning_rate = 0.001
training_epochs = 5

criterion = torch.nn.CrossEntropyLoss().to(device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training myNN

In [7]:
total_batch = len(train_loader)
print('Learning Started!')

for epoch in range(training_epochs):
    running_loss = 0.0
    for data, labels in train_loader:
        data = data.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() / total_batch
    print('[Epoch: {:>4}] loss = {:>.9}'.format(epoch+1, running_loss))

print('Learning Finished!')

Learning Started!
[Epoch:    1] loss = 0.301102183
[Epoch:    2] loss = 0.111385318
[Epoch:    3] loss = 0.0644315397
[Epoch:    4] loss = 0.0446795849
[Epoch:    5] loss = 0.0310100494
Learning Finished!


# Validate myNN

In [8]:
correct = 0
total = 0

model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of myNN on the test set: %.4f %%' % (100 * correct/total))

Accuracy of myNN on the test set: 99.5000 %


# Save the model

In [9]:
torch.save(model.state_dict(), 'model.ckpt')

In [10]:
!ls -lh

total 2.0M
-rw-rw-r-- 1 chris chris 8.1K Oct 25 00:53 MNIST.ipynb
-rw-rw-r-- 1 chris chris 2.0M Oct 25 00:54 model.ckpt
